<a href="https://colab.research.google.com/github/QiaoLin22/MASTER-LLM-DL/blob/main/OpenAI__Distillation_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install --upgrade pip -q
!pip install openai -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.7 MB/s eta 0:00:00


In [2]:
!pip show openai

Name: openai
Version: 1.59.9
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [8]:
import os
import getpass
from openai import OpenAI

from google.colab import userdata


# Set OpenAI API key
api_key = userdata.get('OPENAI_API_KEY')

# Initialize OpenAI client
client = OpenAI(api_key=api_key)

Set up Questions and Evaluate Performance

In [4]:
# Define the training questions
train_questions = [
    "Is a try worth one or five points in touch rugby?",
    "How much is a try worth in touch rugby?",
    "Is a try worth five points in touch rugby?",
    "What happens after a try is scored in touch rugby?",
    "How is the game restarted after a try in touch rugby?",
    "What is the role of the referee in touch rugby?",
    "Are there any differences between touch rugby and rugby union?",
    "What is the maximum number of players on each team in touch rugby?",
    "How long is a typical touch rugby match?",
    "How many players on the field in touch rugby?"
]

In [5]:
import openai

# Function to call OpenAI and print the response
def get_openai_response(model_name, question):
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a sports rules expert."},
            {"role": "user", "content": question}
        ],
        temperature=0.25,
        top_p=0.9 # help avoid bad tokens when sampling.
    )
    print(response.choices[0].message.content)
    print()

def evaluate(model_name,qs_to_eval=10):
  print(f"Generating with model: {model_name}:")
  # Call the function with both models
  for question in train_questions[:qs_to_eval]:
      print("---")
      get_openai_response(model_name, question)

In [9]:
model_name = "gpt-4o" # teacher
evaluate(model_name,1)

Generating with model: gpt-4o:
---
In touch rugby, a try is typically worth one point. Touch rugby is a minimal-contact version of rugby, and its scoring system is simplified compared to traditional rugby union or rugby league, where a try is worth five or four points, respectively. However, it's important to note that specific leagues or tournaments might have variations in their rules, so it's always a good idea to check the specific regulations for the competition you are interested in.



In [10]:
model_name = "gpt-4o-mini" # student
evaluate(model_name,1)

Generating with model: gpt-4o-mini:
---
In touch rugby, a try is worth five points.



Generate and Store Data

In [11]:
dataset_tag = "touch-rugby-7"

# Function to call OpenAI and print the response
def get_and_store_openai_response(model_name, question, store=True):
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a sports rules expert."},
            {"role": "user", "content": question}
        ],
        temperature=0.25,
        top_p=0.9,
        metadata={
            "distillation": dataset_tag
        },
        store=store  # Save the completion
    )
    print(f"Response from {model_name} for '{question}':")
    print(response.choices[0].message.content)
    if store:
      print("Response stored on OpenAI servers")
    print()

# Function to generate data and print question numbers
def generate_data(model_name, qs_to_eval=10):
    # Call the function with both models
    for question_number, question in enumerate(train_questions[:qs_to_eval], start=1):
        print(f"Question {question_number}:")
        print("---")
        get_and_store_openai_response(model_name, question)

In [12]:
model_name = "gpt-4o"
generate_data(model_name)

Question 1:
---
Response from gpt-4o for 'Is a try worth one or five points in touch rugby?':
In touch rugby, a try is typically worth one point. This differs from traditional rugby union, where a try is worth five points. Touch rugby is a minimal-contact version of rugby, and its scoring system reflects the simplified nature of the game.
Response stored on OpenAI servers

Question 2:
---
Response from gpt-4o for 'How much is a try worth in touch rugby?':
In touch rugby, a try is typically worth one point. Unlike other forms of rugby, such as rugby union or rugby league, there are no conversions or additional points for kicks in touch rugby. The primary objective is to score tries, and each try contributes one point to the team's score.
Response stored on OpenAI servers

Question 3:
---
Response from gpt-4o for 'Is a try worth five points in touch rugby?':
In touch rugby, a try is typically worth one point. This differs from traditional rugby union, where a try is worth five points. To